# Economic Freedom and Happiness Across Europe

In [1]:
import pandas as pd
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import pycountry 

from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
